In [8]:
!git clone https://github.com/drryodino246/csc542-gp-team164.git

!pip install datasets

from datasets import load_dataset
emotions = load_dataset('csv', data_files='/content/csc542-gp-team164/baseline.csv')

emotions_split = emotions['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = emotions_split['train']
test_dataset = emotions_split['test']

from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=512, return_tensors="pt")

# Tokenize both the train and test datasets
train_encoded = train_dataset.map(tokenize, batched=True)
test_encoded = test_dataset.map(tokenize, batched=True)

# Rename 'updated_emotion' to 'label' for both train and test datasets
train_encoded = train_encoded.rename_column("updated_emotion", "label")
test_encoded = test_encoded.rename_column("updated_emotion", "label")

# Map string labels to integer labels
def map_labels(example):
    label_map = {
        "very negative": 0,
        "negative": 1,
        "positive": 2,
        "very positive": 3
    }
    example["label"] = label_map[example["label"]]  # Replace string label with integer label
    return example

# Apply the label mapping function to both train and test datasets
train_encoded = train_encoded.map(map_labels)
test_encoded = test_encoded.map(map_labels)

from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 4
id2label = {
    "0": "very negative",
    "1": "negative",
    "2": "positive",
    "3": "very positive"
}
label2id = {
    "very negative": 0,
    "negative": 1,
    "positive": 2,
    "very positive": 3
}

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id).to(device)

for param in model.base_model.parameters():
    param.requires_grad = False

!pip install tensorboard

from transformers import TrainingArguments
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter

batch_size = 64
logging_steps = 500
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f"{model_name}/logs",  # Directory for logs
    logging_steps=logging_steps,  # Log training progress every 'logging_steps'
    load_best_model_at_end=True,  # Load the best model based on evaluation metrics
    metric_for_best_model="f1",  # Track the F1 score for selecting the best model
    greater_is_better=True,  # We want higher F1 score, so set this to True
    disable_tqdm=False,  # Keep TQDM for progress bar
    report_to="tensorboard",  # Enable TensorBoard logging
)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_encoded,
    eval_dataset=test_encoded,
    tokenizer=tokenizer
)

trainer.train()

fatal: destination path 'csc542-gp-team164' already exists and is not an empty directory.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<ipython-input-8-c51026e361bb>:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.309801,0.514464,0.496781,0.524988,0.514464
2,No log,1.282962,0.518634,0.497943,0.525188,0.518634


TrainOutput(global_step=480, training_loss=1.3245316823323567, metrics={'train_runtime': 135.6388, 'train_samples_per_second': 226.292, 'train_steps_per_second': 3.539, 'total_flos': 1000045513686240.0, 'train_loss': 1.3245316823323567, 'epoch': 2.0})